In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.utils import shuffle
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

### CV for old features

In [14]:
train_set = pd.read_csv('data/train_nx_ig_v3.csv')
X = train_set.iloc[:,2:-1]
y = train_set.iloc[:,-1]

In [ ]:
X, y = shuffle(X, y, random_state=0)
X.head()

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [19]:
clf_xgb = XGBClassifier(    
                        n_estimators=200,
                        max_depth=3,
                        objective="binary:logistic",
                        learning_rate=.3, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=5,
                        reg_alpha=2,
                        reg_lambda=500,
                     )
np.mean(cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1'))

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.99532123013673

This result is overfitting because of jaccard coefficient

In [20]:
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=5)

[0]	validation_0-logloss:0.471532
Will train until validation_0-logloss hasn't improved in 5 rounds.
[1]	validation_0-logloss:0.326991
[2]	validation_0-logloss:0.237644
[3]	validation_0-logloss:0.187202
[4]	validation_0-logloss:0.152868
[5]	validation_0-logloss:0.128762
[6]	validation_0-logloss:0.101147
[7]	validation_0-logloss:0.082165
[8]	validation_0-logloss:0.067896
[9]	validation_0-logloss:0.058138
[10]	validation_0-logloss:0.05059
[11]	validation_0-logloss:0.045376
[12]	validation_0-logloss:0.041136
[13]	validation_0-logloss:0.039062
[14]	validation_0-logloss:0.03721
[15]	validation_0-logloss:0.03402
[16]	validation_0-logloss:0.031813
[17]	validation_0-logloss:0.031123
[18]	validation_0-logloss:0.029569
[19]	validation_0-logloss:0.029001
[20]	validation_0-logloss:0.027655
[21]	validation_0-logloss:0.026595
[22]	validation_0-logloss:0.025749
[23]	validation_0-logloss:0.024974
[24]	validation_0-logloss:0.024335
[25]	validation_0-logloss:0.02364
[26]	validation_0-logloss:0.023218
[2

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=5, learning_rate=0.3, max_delta_step=0,
       max_depth=3, min_child_weight=6, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=2, reg_lambda=500, scale_pos_weight=1.6, seed=None,
       silent=True, subsample=0.8)

### Train, predict and output

In [21]:
test_set = pd.read_csv('data/test_nx_ig_v3.csv')
X_test = test_set.iloc[:,2:]
y_test = clf_xgb.predict(X_test)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
df = pd.DataFrame(y_test, columns=['category'])
df.index.name = 'id'
df.to_csv('result/xgb3.csv', index=True, header=True)

### with new features of Shiwen

In [2]:
train_set = pd.read_csv('data/train_treated.csv')
train_set.head()

,id1,id2,link,rno1,rno2,sim,year1,year2,year_diff,common_authors,cn,aai,title_overlap
0,9510123,9502114,1,16827,15446,0.064373,1995,1995,0,0,1,0.513898,2
1,9707075,9604178,1,21154,18059,0.021211,1997,1996,1,0,20,4.320366,1
2,9312155,9506142,0,13074,16171,0.017202,1993,1995,-2,0,0,0.000000,0
3,9911255,302165,0,27486,9702,0.012634,1999,2003,-4,0,0,0.000000,0
4,9701033,209076,0,19856,8212,0.059588,1997,2002,-5,0,0,0.000000,0


In [3]:
X = train_set.loc[:,['sim', 'year_diff', 'common_authors', 'cn', 'aai', 'title_overlap']]
y = train_set.loc[:,'link']
X.head()

,sim,year_diff,common_authors,cn,aai,title_overlap
0,0.064373,0,0,1,0.513898,2
1,0.021211,1,0,20,4.320366,1
2,0.017202,-2,0,0,0.000000,0
3,0.012634,-4,0,0,0.000000,0
4,0.059588,-5,0,0,0.000000,0


In [4]:
X, y = shuffle(X, y, random_state=0)
X.head()

,sim,year_diff,common_authors,cn,aai,title_overlap
206307,0.057576,-1,0,0,0.000000,1
545369,0.064932,6,0,15,3.795183,0
304493,0.009646,0,0,0,0.000000,0
42171,0.002130,-2,0,0,0.000000,0
268996,0.095356,0,0,0,0.000000,0


In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [11]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9734158687953569


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9735239789593948


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


This result is not overfitting.

### replace 5 old features with 5 new ones

In [10]:
train1 = pd.read_csv('data/train_treated.csv')
train1.head()

,id1,id2,link,rno1,rno2,sim,year1,year2,year_diff,common_authors,cn,aai,title_overlap
0,9510123,9502114,1,16827,15446,0.064373,1995,1995,0,0,1,0.513898,2
1,9707075,9604178,1,21154,18059,0.021211,1997,1996,1,0,20,4.320366,1
2,9312155,9506142,0,13074,16171,0.017202,1993,1995,-2,0,0,0.000000,0
3,9911255,302165,0,27486,9702,0.012634,1999,2003,-4,0,0,0.000000,0
4,9701033,209076,0,19856,8212,0.059588,1997,2002,-5,0,0,0.000000,0


In [11]:
train2 = pd.read_csv('data/train_nx_ig_v3.csv')
train2.head()

,id1,id2,res_allo_ind,overlap_title,temp_diff,comm_auth,reduced_tfidf_sim,jaccard_coeff,tgt_citation,labels
0,9510123,9502114,0.142857,2,0,0,0.242161,0.176471,8,1
1,9707075,9604178,0.226401,1,1,0,0.116740,0.106796,124,1
2,9312155,9506142,0.000000,0,-2,0,0.067705,0.000000,2,0
3,9911255,302165,0.000000,0,-4,0,0.054771,0.000000,2,0
4,9701033,209076,0.000000,0,-5,0,0.220903,0.000000,2,0


In [12]:
X = pd.concat([train1.loc[:,['title_overlap', 'year_diff','common_authors','sim','cn','aai']], train2.loc[:,['jaccard_coeff','tgt_citation']]], axis=1)
y = train1.loc[:,'link']
X.head()

,title_overlap,year_diff,common_authors,sim,cn,aai,jaccard_coeff,tgt_citation
0,2,0,0,0.064373,1,0.513898,0.176471,8
1,1,1,0,0.021211,20,4.320366,0.106796,124
2,0,-2,0,0.017202,0,0.000000,0.000000,2
3,0,-4,0,0.012634,0,0.000000,0.000000,2
4,0,-5,0,0.059588,0,0.000000,0.000000,2


In [13]:
X, y = shuffle(X, y, random_state=0)
X.head()

,title_overlap,year_diff,common_authors,sim,cn,aai,jaccard_coeff,tgt_citation
206307,1,-1,0,0.057576,0,0.000000,0.000000,8
545369,0,6,0,0.064932,15,3.795183,0.079439,164
304493,0,0,0,0.009646,0,0.000000,0.000000,0
42171,0,-2,0,0.002130,0,0.000000,0.000000,2
268996,0,0,0,0.095356,0,0.000000,0.030303,14


In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [15]:
clf_xgb = XGBClassifier(    
                        n_estimators=200,
                        max_depth=3,
                        objective="binary:logistic",
                        learning_rate=.3, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=5,
                        reg_alpha=2,
                        reg_lambda=5,
                     )
np.mean(cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1'))

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9960641509156408

This result is overfitting because of jaccard coefficient

### Trials of features

#### 6 new features + tgt citation

In [40]:
train1 = pd.read_csv('data/train_treated.csv')
train2 = pd.read_csv('data/train_nx_ig_v2.csv')
X = pd.concat([train1.loc[:,['title_overlap', 'year_diff','common_authors','sim','cn','aai']], train2.loc[:,['tgt_citation']]], axis=1)
y = train1.loc[:,'link']
X, y = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [ ]:
params_range = {'n_estimators':[100,200,300,400,500], 'max_depth':[3,4], 'learning_rate':[0.1,0.2,0.3,0.4], 'subsample':[0.6,0.7,0.8,0.9,1]}
clf_xgb = XGBClassifier(booster='gbtree',objective="binary:logistic",random_state=0)
clf = GridSearchCV(clf_xgb, param_grid=params_range, scoring='f1',cv=5)
clf.fit(X,y)
print (clf.best_score_)
print (clf.best_estimator_)

In [18]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.4, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9750323735926031


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751365966722221


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.3, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9752286153844733


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753423131238748


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9752185936849571


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753518900772937


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9752188923960535


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753581259631107


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


lower learning rate is better

In [22]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751720296182228


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753447668489581


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751846764675779


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.975384521653423


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=.8,
                        colsample_bylevel=.8,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.975158204280947


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753528063249759


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.3, 
                        subsample=.8,
                        colsample_bytree=.8,
                        colsample_bylevel=.8,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751780846533524


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753319678716019


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
clf_xgb = XGBClassifier(    
                        n_estimators=500,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751650045688935


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753250775885473


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [24]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=50)
y_pred = clf_xgb.predict(X)
f1_score(y,y_pred)

[0]	validation_0-logloss:0.531966
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.423995
[2]	validation_0-logloss:0.346683
[3]	validation_0-logloss:0.289713
[4]	validation_0-logloss:0.246416
[5]	validation_0-logloss:0.213085
[6]	validation_0-logloss:0.187241
[7]	validation_0-logloss:0.166972
[8]	validation_0-logloss:0.150982
[9]	validation_0-logloss:0.138248
[10]	validation_0-logloss:0.127881
[11]	validation_0-logloss:0.119555
[12]	validation_0-logloss:0.112934
[13]	validation_0-logloss:0.107539
[14]	validation_0-logloss:0.103228
[15]	validation_0-logloss:0.099596
[16]	validation_0-logloss:0.096725
[17]	validation_0-logloss:0.09433
[18]	validation_0-logloss:0.092427
[19]	validation_0-logloss:0.090796
[20]	validation_0-logloss:0.089465
[21]	validation_0-logloss:0.088102
[22]	validation_0-logloss:0.087161
[23]	validation_0-logloss:0.086252
[24]	validation_0-logloss:0.085604
[25]	validation_0-logloss:0.085008
[26]	validation_0-logloss:0.08449

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9757823039220848

In [25]:
test1 = pd.read_csv('data/test_treated.csv')
test2 = pd.read_csv('data/test_nx_ig_v2.csv')
X_test = pd.concat([test1.loc[:,['title_overlap','year_diff','common_authors','sim','cn','aai']], test2.loc[:,['tgt_citation']]], axis=1)

y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('result/xgb_6new+tgt.csv', index=True, header=True)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [41]:
clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=50)
y_pred = clf_xgb.predict(X)
f1_score(y,y_pred)

[0]	validation_0-logloss:0.608258
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.538695
[2]	validation_0-logloss:0.48083
[3]	validation_0-logloss:0.431826
[4]	validation_0-logloss:0.390056
[5]	validation_0-logloss:0.354182
[6]	validation_0-logloss:0.323214
[7]	validation_0-logloss:0.29624
[8]	validation_0-logloss:0.27268
[9]	validation_0-logloss:0.252101
[10]	validation_0-logloss:0.23391
[11]	validation_0-logloss:0.217944
[12]	validation_0-logloss:0.203805
[13]	validation_0-logloss:0.191321
[14]	validation_0-logloss:0.18022
[15]	validation_0-logloss:0.170361
[16]	validation_0-logloss:0.161544
[17]	validation_0-logloss:0.15371
[18]	validation_0-logloss:0.146734
[19]	validation_0-logloss:0.140453
[20]	validation_0-logloss:0.134857
[21]	validation_0-logloss:0.129779
[22]	validation_0-logloss:0.125277
[23]	validation_0-logloss:0.121218
[24]	validation_0-logloss:0.117582
[25]	validation_0-logloss:0.114339
[26]	validation_0-logloss:0.111437
[27

[230]	validation_0-logloss:0.079578
[231]	validation_0-logloss:0.079579
[232]	validation_0-logloss:0.07958
[233]	validation_0-logloss:0.079581
[234]	validation_0-logloss:0.079583
[235]	validation_0-logloss:0.079587
[236]	validation_0-logloss:0.079586
[237]	validation_0-logloss:0.079587
[238]	validation_0-logloss:0.079584
[239]	validation_0-logloss:0.079582
[240]	validation_0-logloss:0.079583
[241]	validation_0-logloss:0.079583
[242]	validation_0-logloss:0.079583
[243]	validation_0-logloss:0.079578
[244]	validation_0-logloss:0.079579
[245]	validation_0-logloss:0.079574
[246]	validation_0-logloss:0.079571
[247]	validation_0-logloss:0.079571
[248]	validation_0-logloss:0.079565
[249]	validation_0-logloss:0.079565
[250]	validation_0-logloss:0.079566
[251]	validation_0-logloss:0.079563
[252]	validation_0-logloss:0.079563
[253]	validation_0-logloss:0.079561
[254]	validation_0-logloss:0.079561
[255]	validation_0-logloss:0.079553
[256]	validation_0-logloss:0.079555
[257]	validation_0-logloss:0.

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9757117353422903

In [42]:
test1 = pd.read_csv('data/test_treated.csv')
test2 = pd.read_csv('data/test_nx_ig_v2.csv')
X_test = pd.concat([test1.loc[:,['title_overlap','year_diff','common_authors','sim','cn','aai']], test2.loc[:,['tgt_citation']]], axis=1)

y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('result/xgb_6new+tgt_2.csv', index=True, header=True)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### 6 new features + tgt citation + rai

In [34]:
train1 = pd.read_csv('data/train_treated.csv')
train2 = pd.read_csv('data/train_nx_ig_v2.csv')
X = pd.concat([train1.loc[:,['title_overlap', 'year_diff','common_authors','sim','cn','aai']], train2.loc[:,['res_allo_ind','tgt_citation']]], axis=1)
y = train1.loc[:,'link']
X, y = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [29]:
clf_xgb = XGBClassifier(    
                        n_estimators=500,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751800782293341


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753527316423991


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [30]:
params_range = {'n_estimators':[100,200,300,400,500], 'max_depth':[3,4], 'learning_rate':[0.1,0.2,0.3,0.4], 'subsample':[0.6,0.7,0.8,0.9,1]}
clf_xgb = XGBClassifier(booster='gbtree',objective="binary:logistic",random_state=0)
clf = GridSearchCV(clf_xgb, param_grid=params_range, scoring='f1',cv=5)
clf.fit(X,y)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

...

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 4], 'n_estimators': [100, 200, 300, 400, 500], 'learning_rate': [0.1, 0.2, 0.3, 0.4], 'subsample': [0.6, 0.7, 0.8, 0.9, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=0)

In [31]:
print (clf.best_score_)
print (clf.best_estimator_)

0.9754683031584271
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


In [38]:
clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=50)
y_pred = clf_xgb.predict(X)
f1_score(y,y_pred)

[0]	validation_0-logloss:0.607958
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.538193
[2]	validation_0-logloss:0.480086
[3]	validation_0-logloss:0.431064
[4]	validation_0-logloss:0.3891
[5]	validation_0-logloss:0.353143
[6]	validation_0-logloss:0.321954
[7]	validation_0-logloss:0.29481
[8]	validation_0-logloss:0.271187
[9]	validation_0-logloss:0.250503
[10]	validation_0-logloss:0.232266
[11]	validation_0-logloss:0.216178
[12]	validation_0-logloss:0.201967
[13]	validation_0-logloss:0.189357
[14]	validation_0-logloss:0.178192
[15]	validation_0-logloss:0.168204
[16]	validation_0-logloss:0.159381
[17]	validation_0-logloss:0.151438
[18]	validation_0-logloss:0.14443
[19]	validation_0-logloss:0.138054
[20]	validation_0-logloss:0.132429
[21]	validation_0-logloss:0.127419
[22]	validation_0-logloss:0.122854
[23]	validation_0-logloss:0.118815
[24]	validation_0-logloss:0.115173
[25]	validation_0-logloss:0.111863
[26]	validation_0-logloss:0.108918
[

[230]	validation_0-logloss:0.079243
[231]	validation_0-logloss:0.079246
[232]	validation_0-logloss:0.079251
[233]	validation_0-logloss:0.07925
[234]	validation_0-logloss:0.079252
[235]	validation_0-logloss:0.079253
[236]	validation_0-logloss:0.079252
[237]	validation_0-logloss:0.079251
[238]	validation_0-logloss:0.079252
[239]	validation_0-logloss:0.079252
[240]	validation_0-logloss:0.079252
[241]	validation_0-logloss:0.079249
[242]	validation_0-logloss:0.079249
[243]	validation_0-logloss:0.079251
[244]	validation_0-logloss:0.079251
[245]	validation_0-logloss:0.079253
[246]	validation_0-logloss:0.079254
[247]	validation_0-logloss:0.079253
[248]	validation_0-logloss:0.079251
[249]	validation_0-logloss:0.079251
[250]	validation_0-logloss:0.079253
[251]	validation_0-logloss:0.079255
[252]	validation_0-logloss:0.079249
[253]	validation_0-logloss:0.079248
[254]	validation_0-logloss:0.07925
[255]	validation_0-logloss:0.079247
[256]	validation_0-logloss:0.079247
[257]	validation_0-logloss:0.0

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9756416755542786

In [39]:
test1 = pd.read_csv('data/test_treated.csv')
test2 = pd.read_csv('data/test_nx_ig_v2.csv')
X_test = pd.concat([test1.loc[:,['title_overlap','year_diff','common_authors','sim','cn','aai']], test2.loc[:,['res_allo_ind','tgt_citation']]], axis=1)

y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('result/xgb_6new+tgt+rai.csv', index=True, header=True)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### treated length problem and adding journal overlapping and wmd

In [2]:
train1 = pd.read_csv('data/train_treated_with_journal_wmd.csv')
train2 = pd.read_csv('data/train_nx_ig_v2.csv')
X = pd.concat([train1.loc[:,['title_overlap', 'year_diff','common_authors','sim','cn','aai','journal_overlap','wmd']], train2.loc[:,['res_allo_ind','tgt_citation']]], axis=1)
y = train1.loc[:,'link']
X, y = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [3]:
clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753943984855356


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.975751899489316


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### new new features

In [2]:
train1 = pd.read_csv('../data/train_stacked.csv')
train2 = pd.read_csv('../data/train_nx_ig_v2.csv')
X = pd.concat([train1.loc[:,['sim','cn','aai','year_diff','common_authors','title_overlap','journal_overlap','wmd']], train2.loc[:,['res_allo_ind','tgt_citation','jaccard_nx']]], axis=1)
y = train1.loc[:,'link']

In [3]:
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)
clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_cv, y_cv, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9758031095082211


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### to stack

In [4]:
clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=80)
y_pred = clf_xgb.predict(X)
print (f1_score(y,y_pred))

train1['xgb_stack'] = y_pred
train1.to_csv('../data/train_stacked.csv', index=False)

[0]	validation_0-logloss:0.608115
Will train until validation_0-logloss hasn't improved in 80 rounds.
[1]	validation_0-logloss:0.538405
[2]	validation_0-logloss:0.480291
[3]	validation_0-logloss:0.431149
[4]	validation_0-logloss:0.389162
[5]	validation_0-logloss:0.352994
[6]	validation_0-logloss:0.321813
[7]	validation_0-logloss:0.294727
[8]	validation_0-logloss:0.271017
[9]	validation_0-logloss:0.250266
[10]	validation_0-logloss:0.232003
[11]	validation_0-logloss:0.215905
[12]	validation_0-logloss:0.20167
[13]	validation_0-logloss:0.18905
[14]	validation_0-logloss:0.177775
[15]	validation_0-logloss:0.167769
[16]	validation_0-logloss:0.15892
[17]	validation_0-logloss:0.151023
[18]	validation_0-logloss:0.143969
[19]	validation_0-logloss:0.13767
[20]	validation_0-logloss:0.13194
[21]	validation_0-logloss:0.126891
[22]	validation_0-logloss:0.122381
[23]	validation_0-logloss:0.118257
[24]	validation_0-logloss:0.114644
[25]	validation_0-logloss:0.111371
[26]	validation_0-logloss:0.108438
[2

[230]	validation_0-logloss:0.078093
[231]	validation_0-logloss:0.078094
[232]	validation_0-logloss:0.078093
[233]	validation_0-logloss:0.078089
[234]	validation_0-logloss:0.078086
[235]	validation_0-logloss:0.07809
[236]	validation_0-logloss:0.078086
[237]	validation_0-logloss:0.078086
[238]	validation_0-logloss:0.078079
[239]	validation_0-logloss:0.078077
[240]	validation_0-logloss:0.078077
[241]	validation_0-logloss:0.078075
[242]	validation_0-logloss:0.078075
[243]	validation_0-logloss:0.078074
[244]	validation_0-logloss:0.078075
[245]	validation_0-logloss:0.078076
[246]	validation_0-logloss:0.078077
[247]	validation_0-logloss:0.078079
[248]	validation_0-logloss:0.078082
[249]	validation_0-logloss:0.078079
[250]	validation_0-logloss:0.078083
[251]	validation_0-logloss:0.078082
[252]	validation_0-logloss:0.078083
[253]	validation_0-logloss:0.07808
[254]	validation_0-logloss:0.078084
[255]	validation_0-logloss:0.078084
[256]	validation_0-logloss:0.078082
[257]	validation_0-logloss:0.0

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9762191609399229


In [5]:
test1 = pd.read_csv('../data/test_stacked.csv')
test2 = pd.read_csv('../data/test_nx_ig_v2.csv')
X_test = pd.concat([test1.loc[:,['sim','cn','aai','year_diff','common_authors','title_overlap','journal_overlap','wmd']], test2.loc[:,['res_allo_ind','tgt_citation','jaccard_nx']]], axis=1)

y_test_pred = clf_xgb.predict(X_test)
test1['xgb_stack'] = y_test_pred
test1.to_csv('../data/test_stacked.csv', index=False)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
train1.head()

,id1,id2,link,rno1,rno2,sim,cn,aai,year1,year2,year_diff,common_authors,title_overlap,journal_overlap,centroid_sim,wmd,rf_stack,lr_stack,xgb_stack
0,9510123,9502114,1,16827,15446,0.064373,1,0.513898,1995,1995,0,0.0,0.285714,1.0,0.683964,1.872725,1,1,1
1,9707075,9604178,1,21154,18059,0.021211,20,4.320366,1997,1996,1,0.0,0.250000,0.0,0.544016,1.335541,1,1,1
2,9312155,9506142,0,13074,16171,0.017202,0,0.000000,1993,1995,-2,0.0,0.000000,0.0,0.761403,2.087327,0,0,0
3,9911255,302165,0,27486,9702,0.012634,0,0.000000,1999,2003,-4,0.0,0.000000,0.0,0.738870,1.602484,0,0,0
4,9701033,209076,0,19856,8212,0.059588,0,0.000000,1997,2002,-5,0.0,0.000000,0.0,0.733977,1.414463,0,0,0


In [12]:
y_test_pred = pd.read_csv('../data/test_stacked.csv').loc[:, 'xgb_stack'].values
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../result/xgb_new.csv', index=True, header=True)

### combined features

In [2]:
# train_all
train = pd.read_csv('../../project1/data/train_all.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=80)
y_pred = clf_xgb.predict(X)
print (f1_score(y,y_pred))

[0]	validation_0-logloss:0.607785
Will train until validation_0-logloss hasn't improved in 80 rounds.
[1]	validation_0-logloss:0.537965
[2]	validation_0-logloss:0.47965
[3]	validation_0-logloss:0.430446
[4]	validation_0-logloss:0.388367
[5]	validation_0-logloss:0.352221
[6]	validation_0-logloss:0.3209
[7]	validation_0-logloss:0.293725
[8]	validation_0-logloss:0.269932
[9]	validation_0-logloss:0.249059
[10]	validation_0-logloss:0.23071
[11]	validation_0-logloss:0.214415
[12]	validation_0-logloss:0.200128
[13]	validation_0-logloss:0.187342
[14]	validation_0-logloss:0.176055
[15]	validation_0-logloss:0.166055
[16]	validation_0-logloss:0.157107
[17]	validation_0-logloss:0.149167
[18]	validation_0-logloss:0.142126
[19]	validation_0-logloss:0.135752
[20]	validation_0-logloss:0.130075
[21]	validation_0-logloss:0.124989
[22]	validation_0-logloss:0.120403
[23]	validation_0-logloss:0.116307
[24]	validation_0-logloss:0.112556
[25]	validation_0-logloss:0.109252
[26]	validation_0-logloss:0.106277
[

[230]	validation_0-logloss:0.074539
[231]	validation_0-logloss:0.07454
[232]	validation_0-logloss:0.074539
[233]	validation_0-logloss:0.074538
[234]	validation_0-logloss:0.074537
[235]	validation_0-logloss:0.07453
[236]	validation_0-logloss:0.074527
[237]	validation_0-logloss:0.074503
[238]	validation_0-logloss:0.074499
[239]	validation_0-logloss:0.074499
[240]	validation_0-logloss:0.0745
[241]	validation_0-logloss:0.074492
[242]	validation_0-logloss:0.074492
[243]	validation_0-logloss:0.074491
[244]	validation_0-logloss:0.074484
[245]	validation_0-logloss:0.074477
[246]	validation_0-logloss:0.074478
[247]	validation_0-logloss:0.074477
[248]	validation_0-logloss:0.074475
[249]	validation_0-logloss:0.074475
[250]	validation_0-logloss:0.074482
[251]	validation_0-logloss:0.07447
[252]	validation_0-logloss:0.074472
[253]	validation_0-logloss:0.074464
[254]	validation_0-logloss:0.074458
[255]	validation_0-logloss:0.074456
[256]	validation_0-logloss:0.074456
[257]	validation_0-logloss:0.0744

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [3]:
# train_all, drop pa worse!
train = pd.read_csv('../../project1/data/train_all.csv')
X = train.iloc[:, 5:]
X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=80)
y_pred = clf_xgb.predict(X)
print (f1_score(y,y_pred))

[0]	validation_0-logloss:0.607785
Will train until validation_0-logloss hasn't improved in 80 rounds.
[1]	validation_0-logloss:0.537965
[2]	validation_0-logloss:0.47965
[3]	validation_0-logloss:0.430446
[4]	validation_0-logloss:0.388367
[5]	validation_0-logloss:0.352221
[6]	validation_0-logloss:0.3209
[7]	validation_0-logloss:0.293725
[8]	validation_0-logloss:0.269932
[9]	validation_0-logloss:0.249059
[10]	validation_0-logloss:0.23071
[11]	validation_0-logloss:0.214415
[12]	validation_0-logloss:0.200128
[13]	validation_0-logloss:0.187342
[14]	validation_0-logloss:0.176055
[15]	validation_0-logloss:0.166055
[16]	validation_0-logloss:0.157107
[17]	validation_0-logloss:0.149167
[18]	validation_0-logloss:0.142126
[19]	validation_0-logloss:0.135752
[20]	validation_0-logloss:0.130075
[21]	validation_0-logloss:0.124989
[22]	validation_0-logloss:0.120403
[23]	validation_0-logloss:0.116307
[24]	validation_0-logloss:0.112556
[25]	validation_0-logloss:0.109252
[26]	validation_0-logloss:0.106277
[

[230]	validation_0-logloss:0.074985
[231]	validation_0-logloss:0.074991
[232]	validation_0-logloss:0.07499
[233]	validation_0-logloss:0.074983
[234]	validation_0-logloss:0.074975
[235]	validation_0-logloss:0.074975
[236]	validation_0-logloss:0.074972
[237]	validation_0-logloss:0.074972
[238]	validation_0-logloss:0.074966
[239]	validation_0-logloss:0.074966
[240]	validation_0-logloss:0.074962
[241]	validation_0-logloss:0.074952
[242]	validation_0-logloss:0.074953
[243]	validation_0-logloss:0.07495
[244]	validation_0-logloss:0.07495
[245]	validation_0-logloss:0.07495
[246]	validation_0-logloss:0.07494
[247]	validation_0-logloss:0.074935
[248]	validation_0-logloss:0.074939
[249]	validation_0-logloss:0.074937
[250]	validation_0-logloss:0.074936
[251]	validation_0-logloss:0.074935
[252]	validation_0-logloss:0.074936
[253]	validation_0-logloss:0.074939
[254]	validation_0-logloss:0.074939
[255]	validation_0-logloss:0.074938
[256]	validation_0-logloss:0.074938
[257]	validation_0-logloss:0.0749

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [4]:
# train_all2
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=80)
y_pred = clf_xgb.predict(X)
print (f1_score(y,y_pred))

[0]	validation_0-logloss:0.607785
Will train until validation_0-logloss hasn't improved in 80 rounds.
[1]	validation_0-logloss:0.537965
[2]	validation_0-logloss:0.47965
[3]	validation_0-logloss:0.430446
[4]	validation_0-logloss:0.388367
[5]	validation_0-logloss:0.352221
[6]	validation_0-logloss:0.3209
[7]	validation_0-logloss:0.293726
[8]	validation_0-logloss:0.26994
[9]	validation_0-logloss:0.24905
[10]	validation_0-logloss:0.230729
[11]	validation_0-logloss:0.21443
[12]	validation_0-logloss:0.200126
[13]	validation_0-logloss:0.187353
[14]	validation_0-logloss:0.176053
[15]	validation_0-logloss:0.166057
[16]	validation_0-logloss:0.157104
[17]	validation_0-logloss:0.149176
[18]	validation_0-logloss:0.142133
[19]	validation_0-logloss:0.135759
[20]	validation_0-logloss:0.130087
[21]	validation_0-logloss:0.124936
[22]	validation_0-logloss:0.12037
[23]	validation_0-logloss:0.116278
[24]	validation_0-logloss:0.112634
[25]	validation_0-logloss:0.109255
[26]	validation_0-logloss:0.10626
[27]	

[230]	validation_0-logloss:0.074449
[231]	validation_0-logloss:0.074448
[232]	validation_0-logloss:0.074445
[233]	validation_0-logloss:0.074442
[234]	validation_0-logloss:0.074435
[235]	validation_0-logloss:0.074433
[236]	validation_0-logloss:0.074433
[237]	validation_0-logloss:0.074432
[238]	validation_0-logloss:0.07442
[239]	validation_0-logloss:0.074399
[240]	validation_0-logloss:0.0744
[241]	validation_0-logloss:0.074393
[242]	validation_0-logloss:0.074394
[243]	validation_0-logloss:0.074395
[244]	validation_0-logloss:0.074394
[245]	validation_0-logloss:0.074391
[246]	validation_0-logloss:0.074391
[247]	validation_0-logloss:0.074391
[248]	validation_0-logloss:0.07439
[249]	validation_0-logloss:0.074386
[250]	validation_0-logloss:0.074387
[251]	validation_0-logloss:0.074377
[252]	validation_0-logloss:0.074373
[253]	validation_0-logloss:0.074374
[254]	validation_0-logloss:0.074359
[255]	validation_0-logloss:0.074354
[256]	validation_0-logloss:0.074348
[257]	validation_0-logloss:0.074

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
# train_all2, drop pa worse!
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=80)
y_pred = clf_xgb.predict(X)
print (f1_score(y,y_pred))

[0]	validation_0-logloss:0.607785
Will train until validation_0-logloss hasn't improved in 80 rounds.
[1]	validation_0-logloss:0.537965
[2]	validation_0-logloss:0.47965
[3]	validation_0-logloss:0.430446
[4]	validation_0-logloss:0.388367
[5]	validation_0-logloss:0.352221
[6]	validation_0-logloss:0.3209
[7]	validation_0-logloss:0.293726
[8]	validation_0-logloss:0.26994
[9]	validation_0-logloss:0.24905
[10]	validation_0-logloss:0.230729
[11]	validation_0-logloss:0.21443
[12]	validation_0-logloss:0.200126
[13]	validation_0-logloss:0.187353
[14]	validation_0-logloss:0.176053
[15]	validation_0-logloss:0.166057
[16]	validation_0-logloss:0.157104
[17]	validation_0-logloss:0.149176
[18]	validation_0-logloss:0.142133
[19]	validation_0-logloss:0.135759
[20]	validation_0-logloss:0.130087
[21]	validation_0-logloss:0.124936
[22]	validation_0-logloss:0.12037
[23]	validation_0-logloss:0.116278
[24]	validation_0-logloss:0.112634
[25]	validation_0-logloss:0.109255
[26]	validation_0-logloss:0.10626
[27]	

[230]	validation_0-logloss:0.074895
[231]	validation_0-logloss:0.074898
[232]	validation_0-logloss:0.074898
[233]	validation_0-logloss:0.0749
[234]	validation_0-logloss:0.074897
[235]	validation_0-logloss:0.074883
[236]	validation_0-logloss:0.074881
[237]	validation_0-logloss:0.074881
[238]	validation_0-logloss:0.074884
[239]	validation_0-logloss:0.074885
[240]	validation_0-logloss:0.074879
[241]	validation_0-logloss:0.074878
[242]	validation_0-logloss:0.074879
[243]	validation_0-logloss:0.074879
[244]	validation_0-logloss:0.074877
[245]	validation_0-logloss:0.074874
[246]	validation_0-logloss:0.074869
[247]	validation_0-logloss:0.074869
[248]	validation_0-logloss:0.074869
[249]	validation_0-logloss:0.07487
[250]	validation_0-logloss:0.074866
[251]	validation_0-logloss:0.074866
[252]	validation_0-logloss:0.074865
[253]	validation_0-logloss:0.074864
[254]	validation_0-logloss:0.074868
[255]	validation_0-logloss:0.074854
[256]	validation_0-logloss:0.074846
[257]	validation_0-logloss:0.07

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
# train_all2, not drop pa!
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=50)
y_pred = clf_xgb.predict(X)
print (f1_score(y,y_pred))

test = pd.read_csv('../../project1/data/test_all2.csv')
X_test = test.iloc[:, 4:]
# X_test.drop(labels=['pa'], axis=1, inplace=True)
y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_2.csv', index=True, header=True) # early stopping 50

[0]	validation_0-logloss:0.607785
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.537965
[2]	validation_0-logloss:0.47965
[3]	validation_0-logloss:0.430446
[4]	validation_0-logloss:0.388367
[5]	validation_0-logloss:0.352221
[6]	validation_0-logloss:0.3209
[7]	validation_0-logloss:0.293726
[8]	validation_0-logloss:0.26994
[9]	validation_0-logloss:0.24905
[10]	validation_0-logloss:0.230729
[11]	validation_0-logloss:0.21443
[12]	validation_0-logloss:0.200126
[13]	validation_0-logloss:0.187353
[14]	validation_0-logloss:0.176053
[15]	validation_0-logloss:0.166057
[16]	validation_0-logloss:0.157104
[17]	validation_0-logloss:0.149176
[18]	validation_0-logloss:0.142133
[19]	validation_0-logloss:0.135759
[20]	validation_0-logloss:0.130087
[21]	validation_0-logloss:0.124936
[22]	validation_0-logloss:0.12037
[23]	validation_0-logloss:0.116278
[24]	validation_0-logloss:0.112634
[25]	validation_0-logloss:0.109255
[26]	validation_0-logloss:0.10626
[27]	

[230]	validation_0-logloss:0.074449
[231]	validation_0-logloss:0.074448
[232]	validation_0-logloss:0.074445
[233]	validation_0-logloss:0.074442
[234]	validation_0-logloss:0.074435
[235]	validation_0-logloss:0.074433
[236]	validation_0-logloss:0.074433
[237]	validation_0-logloss:0.074432
[238]	validation_0-logloss:0.07442
[239]	validation_0-logloss:0.074399
[240]	validation_0-logloss:0.0744
[241]	validation_0-logloss:0.074393
[242]	validation_0-logloss:0.074394
[243]	validation_0-logloss:0.074395
[244]	validation_0-logloss:0.074394
[245]	validation_0-logloss:0.074391
[246]	validation_0-logloss:0.074391
[247]	validation_0-logloss:0.074391
[248]	validation_0-logloss:0.07439
[249]	validation_0-logloss:0.074386
[250]	validation_0-logloss:0.074387
[251]	validation_0-logloss:0.074377
[252]	validation_0-logloss:0.074373
[253]	validation_0-logloss:0.074374
[254]	validation_0-logloss:0.074359
[255]	validation_0-logloss:0.074354
[256]	validation_0-logloss:0.074348
[257]	validation_0-logloss:0.074

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9777052598999675


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
# train_all2, not drop pa!
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=3,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=50)
y_pred = clf_xgb.predict(X)
print (f1_score(y,y_pred))

test = pd.read_csv('../../project1/data/test_all2.csv')
X_test = test.iloc[:, 4:]
# X_test.drop(labels=['pa'], axis=1, inplace=True)
y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_3.csv', index=True, header=True) # early stopping 50, max depth 3

[0]	validation_0-logloss:0.608574
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.53921
[2]	validation_0-logloss:0.481222
[3]	validation_0-logloss:0.432346
[4]	validation_0-logloss:0.390505
[5]	validation_0-logloss:0.354736
[6]	validation_0-logloss:0.32366
[7]	validation_0-logloss:0.296816
[8]	validation_0-logloss:0.273205
[9]	validation_0-logloss:0.25262
[10]	validation_0-logloss:0.234438
[11]	validation_0-logloss:0.218466
[12]	validation_0-logloss:0.2042
[13]	validation_0-logloss:0.191703
[14]	validation_0-logloss:0.180494
[15]	validation_0-logloss:0.170627
[16]	validation_0-logloss:0.16175
[17]	validation_0-logloss:0.153896
[18]	validation_0-logloss:0.146784
[19]	validation_0-logloss:0.140495
[20]	validation_0-logloss:0.134794
[21]	validation_0-logloss:0.129777
[22]	validation_0-logloss:0.125243
[23]	validation_0-logloss:0.121076
[24]	validation_0-logloss:0.117453
[25]	validation_0-logloss:0.114095
[26]	validation_0-logloss:0.111111
[27

[230]	validation_0-logloss:0.075049
[231]	validation_0-logloss:0.075045
[232]	validation_0-logloss:0.075047
[233]	validation_0-logloss:0.075045
[234]	validation_0-logloss:0.075037
[235]	validation_0-logloss:0.07503
[236]	validation_0-logloss:0.075008
[237]	validation_0-logloss:0.075003
[238]	validation_0-logloss:0.075002
[239]	validation_0-logloss:0.074999
[240]	validation_0-logloss:0.074997
[241]	validation_0-logloss:0.074983
[242]	validation_0-logloss:0.074982
[243]	validation_0-logloss:0.074981
[244]	validation_0-logloss:0.074979
[245]	validation_0-logloss:0.074975
[246]	validation_0-logloss:0.074973
[247]	validation_0-logloss:0.07497
[248]	validation_0-logloss:0.074968
[249]	validation_0-logloss:0.074967
[250]	validation_0-logloss:0.074962
[251]	validation_0-logloss:0.07496
[252]	validation_0-logloss:0.07495
[253]	validation_0-logloss:0.07495
[254]	validation_0-logloss:0.07495
[255]	validation_0-logloss:0.074951
[256]	validation_0-logloss:0.074946
[257]	validation_0-logloss:0.07494

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9771768396759383


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [11]:
# train_all2, not drop pa!
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=2,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=30)
y_pred = clf_xgb.predict(X)
df = pd.DataFrame(y_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_4_train.csv', index=True, header=True)
print (f1_score(y,y_pred))

test = pd.read_csv('../../project1/data/test_all2.csv')
X_test = test.iloc[:, 4:]
# X_test.drop(labels=['pa'], axis=1, inplace=True)
y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_4.csv', index=True, header=True) # early stopping 30, max depth 2

[0]	validation_0-logloss:0.609767
Will train until validation_0-logloss hasn't improved in 30 rounds.
[1]	validation_0-logloss:0.541394
[2]	validation_0-logloss:0.484501
[3]	validation_0-logloss:0.436527
[4]	validation_0-logloss:0.395554
[5]	validation_0-logloss:0.36049
[6]	validation_0-logloss:0.330136
[7]	validation_0-logloss:0.303809
[8]	validation_0-logloss:0.2808
[9]	validation_0-logloss:0.260536
[10]	validation_0-logloss:0.2429
[11]	validation_0-logloss:0.227397
[12]	validation_0-logloss:0.21356
[13]	validation_0-logloss:0.201414
[14]	validation_0-logloss:0.190516
[15]	validation_0-logloss:0.180892
[16]	validation_0-logloss:0.172348
[17]	validation_0-logloss:0.164751
[18]	validation_0-logloss:0.157842
[19]	validation_0-logloss:0.151664
[20]	validation_0-logloss:0.146156
[21]	validation_0-logloss:0.141251
[22]	validation_0-logloss:0.136747
[23]	validation_0-logloss:0.132659
[24]	validation_0-logloss:0.129003
[25]	validation_0-logloss:0.125741
[26]	validation_0-logloss:0.122722
[27

[230]	validation_0-logloss:0.076008
[231]	validation_0-logloss:0.076002
[232]	validation_0-logloss:0.075995
[233]	validation_0-logloss:0.075989
[234]	validation_0-logloss:0.075984
[235]	validation_0-logloss:0.075982
[236]	validation_0-logloss:0.075975
[237]	validation_0-logloss:0.075965
[238]	validation_0-logloss:0.075953
[239]	validation_0-logloss:0.075941
[240]	validation_0-logloss:0.075929
[241]	validation_0-logloss:0.075928
[242]	validation_0-logloss:0.075921
[243]	validation_0-logloss:0.075916
[244]	validation_0-logloss:0.075906
[245]	validation_0-logloss:0.075897
[246]	validation_0-logloss:0.075893
[247]	validation_0-logloss:0.07588
[248]	validation_0-logloss:0.075876
[249]	validation_0-logloss:0.075864
[250]	validation_0-logloss:0.075862
[251]	validation_0-logloss:0.075858
[252]	validation_0-logloss:0.075837
[253]	validation_0-logloss:0.07583
[254]	validation_0-logloss:0.075829
[255]	validation_0-logloss:0.075823
[256]	validation_0-logloss:0.075815
[257]	validation_0-logloss:0.0

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9766912386125423


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
# train_all2, not drop pa!
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=2,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.8,
                        colsample_bytree=.8,
                        colsample_bylevel=.8,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=15)
y_pred = clf_xgb.predict(X)
df = pd.DataFrame(y_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_5_train.csv', index=True, header=True)
print (f1_score(y,y_pred))

test = pd.read_csv('../../project1/data/test_all2.csv')
X_test = test.iloc[:, 4:]
# X_test.drop(labels=['pa'], axis=1, inplace=True)
y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_5.csv', index=True, header=True) # early stopping 15, max depth 2, subsample 0.8

[0]	validation_0-logloss:0.611607
Will train until validation_0-logloss hasn't improved in 15 rounds.
[1]	validation_0-logloss:0.542809
[2]	validation_0-logloss:0.486483
[3]	validation_0-logloss:0.438053
[4]	validation_0-logloss:0.397072
[5]	validation_0-logloss:0.361792
[6]	validation_0-logloss:0.331362
[7]	validation_0-logloss:0.305563
[8]	validation_0-logloss:0.282383
[9]	validation_0-logloss:0.262164
[10]	validation_0-logloss:0.244248
[11]	validation_0-logloss:0.22856
[12]	validation_0-logloss:0.214875
[13]	validation_0-logloss:0.202481
[14]	validation_0-logloss:0.191507
[15]	validation_0-logloss:0.18172
[16]	validation_0-logloss:0.172987
[17]	validation_0-logloss:0.165227
[18]	validation_0-logloss:0.158432
[19]	validation_0-logloss:0.152373
[20]	validation_0-logloss:0.146769
[21]	validation_0-logloss:0.142015
[22]	validation_0-logloss:0.13758
[23]	validation_0-logloss:0.133297
[24]	validation_0-logloss:0.129727
[25]	validation_0-logloss:0.126234
[26]	validation_0-logloss:0.1232
[2

[230]	validation_0-logloss:0.076182
[231]	validation_0-logloss:0.076174
[232]	validation_0-logloss:0.076158
[233]	validation_0-logloss:0.076147
[234]	validation_0-logloss:0.076135
[235]	validation_0-logloss:0.076129
[236]	validation_0-logloss:0.076119
[237]	validation_0-logloss:0.076115
[238]	validation_0-logloss:0.076109
[239]	validation_0-logloss:0.076096
[240]	validation_0-logloss:0.076074
[241]	validation_0-logloss:0.076059
[242]	validation_0-logloss:0.076052
[243]	validation_0-logloss:0.076046
[244]	validation_0-logloss:0.076045
[245]	validation_0-logloss:0.076033
[246]	validation_0-logloss:0.076017
[247]	validation_0-logloss:0.076014
[248]	validation_0-logloss:0.076011
[249]	validation_0-logloss:0.076006
[250]	validation_0-logloss:0.075997
[251]	validation_0-logloss:0.075981
[252]	validation_0-logloss:0.075973
[253]	validation_0-logloss:0.07597
[254]	validation_0-logloss:0.075946
[255]	validation_0-logloss:0.075939
[256]	validation_0-logloss:0.07594
[257]	validation_0-logloss:0.0

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9766432643192142


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
# train_all2, not drop pa!
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=2,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.6,
                        colsample_bytree=.6,
                        colsample_bylevel=.6,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=15)
y_pred = clf_xgb.predict(X)
df = pd.DataFrame(y_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_6_train.csv', index=True, header=True)
print (f1_score(y,y_pred))

test = pd.read_csv('../../project1/data/test_all2.csv')
X_test = test.iloc[:, 4:]
# X_test.drop(labels=['pa'], axis=1, inplace=True)
y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_6.csv', index=True, header=True) # early stopping 15, max depth 2, subsample 0.6

[0]	validation_0-logloss:0.61187
Will train until validation_0-logloss hasn't improved in 15 rounds.
[1]	validation_0-logloss:0.563019
[2]	validation_0-logloss:0.502557
[3]	validation_0-logloss:0.452006
[4]	validation_0-logloss:0.410599
[5]	validation_0-logloss:0.374001
[6]	validation_0-logloss:0.341779
[7]	validation_0-logloss:0.313859
[8]	validation_0-logloss:0.291026
[9]	validation_0-logloss:0.269502
[10]	validation_0-logloss:0.250946
[11]	validation_0-logloss:0.235033
[12]	validation_0-logloss:0.220206
[13]	validation_0-logloss:0.206777
[14]	validation_0-logloss:0.195343
[15]	validation_0-logloss:0.185702
[16]	validation_0-logloss:0.176494
[17]	validation_0-logloss:0.168203
[18]	validation_0-logloss:0.160686
[19]	validation_0-logloss:0.154269
[20]	validation_0-logloss:0.148758
[21]	validation_0-logloss:0.142958
[22]	validation_0-logloss:0.138346
[23]	validation_0-logloss:0.134028
[24]	validation_0-logloss:0.130429
[25]	validation_0-logloss:0.127084
[26]	validation_0-logloss:0.12407

[230]	validation_0-logloss:0.076502
[231]	validation_0-logloss:0.076494
[232]	validation_0-logloss:0.076483
[233]	validation_0-logloss:0.076477
[234]	validation_0-logloss:0.076456
[235]	validation_0-logloss:0.076453
[236]	validation_0-logloss:0.076441
[237]	validation_0-logloss:0.076432
[238]	validation_0-logloss:0.076428
[239]	validation_0-logloss:0.076414
[240]	validation_0-logloss:0.076404
[241]	validation_0-logloss:0.076396
[242]	validation_0-logloss:0.076375
[243]	validation_0-logloss:0.076371
[244]	validation_0-logloss:0.076353
[245]	validation_0-logloss:0.076348
[246]	validation_0-logloss:0.076319
[247]	validation_0-logloss:0.076304
[248]	validation_0-logloss:0.076286
[249]	validation_0-logloss:0.07628
[250]	validation_0-logloss:0.076272
[251]	validation_0-logloss:0.076265
[252]	validation_0-logloss:0.076262
[253]	validation_0-logloss:0.076258
[254]	validation_0-logloss:0.076249
[255]	validation_0-logloss:0.076224
[256]	validation_0-logloss:0.076216
[257]	validation_0-logloss:0.

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9766102622549536


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
# train_all2, not drop pa!
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=2,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.2,
                        colsample_bytree=.2,
                        colsample_bylevel=.2,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=5)
y_pred = clf_xgb.predict(X)
df = pd.DataFrame(y_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_7_train.csv', index=True, header=True)
print (f1_score(y,y_pred))

test = pd.read_csv('../../project1/data/test_all2.csv')
X_test = test.iloc[:, 4:]
# X_test.drop(labels=['pa'], axis=1, inplace=True)
y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_7.csv', index=True, header=True) # early stopping 5, max depth 2, subsample 0.2

[0]	validation_0-logloss:0.661024
Will train until validation_0-logloss hasn't improved in 5 rounds.
[1]	validation_0-logloss:0.586543
[2]	validation_0-logloss:0.55785
[3]	validation_0-logloss:0.539213
[4]	validation_0-logloss:0.484304
[5]	validation_0-logloss:0.475408
[6]	validation_0-logloss:0.429124
[7]	validation_0-logloss:0.422664
[8]	validation_0-logloss:0.409264
[9]	validation_0-logloss:0.389061
[10]	validation_0-logloss:0.383893
[11]	validation_0-logloss:0.377255
[12]	validation_0-logloss:0.344395
[13]	validation_0-logloss:0.333747
[14]	validation_0-logloss:0.30654
[15]	validation_0-logloss:0.297431
[16]	validation_0-logloss:0.274935
[17]	validation_0-logloss:0.268231
[18]	validation_0-logloss:0.262645
[19]	validation_0-logloss:0.255297
[20]	validation_0-logloss:0.24972
[21]	validation_0-logloss:0.233454
[22]	validation_0-logloss:0.232332
[23]	validation_0-logloss:0.22799
[24]	validation_0-logloss:0.213598
[25]	validation_0-logloss:0.208599
[26]	validation_0-logloss:0.203089
[2

[230]	validation_0-logloss:0.080653
[231]	validation_0-logloss:0.08064
[232]	validation_0-logloss:0.080612
[233]	validation_0-logloss:0.080609
[234]	validation_0-logloss:0.080588
[235]	validation_0-logloss:0.080569
[236]	validation_0-logloss:0.080546
[237]	validation_0-logloss:0.08054
[238]	validation_0-logloss:0.080498
[239]	validation_0-logloss:0.080487
[240]	validation_0-logloss:0.080407
[241]	validation_0-logloss:0.080383
[242]	validation_0-logloss:0.080342
[243]	validation_0-logloss:0.080321
[244]	validation_0-logloss:0.08031
[245]	validation_0-logloss:0.080296
[246]	validation_0-logloss:0.080216
[247]	validation_0-logloss:0.080207
[248]	validation_0-logloss:0.080201
[249]	validation_0-logloss:0.080178
[250]	validation_0-logloss:0.080102
[251]	validation_0-logloss:0.080007
[252]	validation_0-logloss:0.079983
[253]	validation_0-logloss:0.079973
[254]	validation_0-logloss:0.079945
[255]	validation_0-logloss:0.079912
[256]	validation_0-logloss:0.079899
[257]	validation_0-logloss:0.07

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9753519430628982


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
# train_all2, not drop pa!
train = pd.read_csv('../../project1/data/train_all2.csv')
X = train.iloc[:, 5:]
# X.drop(labels=['pa'], axis=1, inplace=True)
y = train.loc[:,'link']
X_cv, y_cv = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_cv, y_cv, test_size=0.3, random_state=0, shuffle=True)

clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=2,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.1,
                        colsample_bytree=.1,
                        colsample_bylevel=.1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=5)
y_pred = clf_xgb.predict(X)
df = pd.DataFrame(y_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_8_train.csv', index=True, header=True)
print (f1_score(y,y_pred))

test = pd.read_csv('../../project1/data/test_all2.csv')
X_test = test.iloc[:, 4:]
# X_test.drop(labels=['pa'], axis=1, inplace=True)
y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('../../project1/result/xgb_all2_8.csv', index=True, header=True) # early stopping 5, max depth 2, subsample 0.1

[0]	validation_0-logloss:0.611617
Will train until validation_0-logloss hasn't improved in 5 rounds.
[1]	validation_0-logloss:0.546799
[2]	validation_0-logloss:0.490946
[3]	validation_0-logloss:0.444211
[4]	validation_0-logloss:0.403921
[5]	validation_0-logloss:0.38817
[6]	validation_0-logloss:0.365382
[7]	validation_0-logloss:0.350409
[8]	validation_0-logloss:0.348556
[9]	validation_0-logloss:0.334672
[10]	validation_0-logloss:0.333598
[11]	validation_0-logloss:0.332124
[12]	validation_0-logloss:0.321697
[13]	validation_0-logloss:0.297021
[14]	validation_0-logloss:0.287435
[15]	validation_0-logloss:0.266852
[16]	validation_0-logloss:0.257023
[17]	validation_0-logloss:0.246253
[18]	validation_0-logloss:0.242769
[19]	validation_0-logloss:0.241499
[20]	validation_0-logloss:0.235541
[21]	validation_0-logloss:0.229108
[22]	validation_0-logloss:0.222741
[23]	validation_0-logloss:0.215338
[24]	validation_0-logloss:0.212902
[25]	validation_0-logloss:0.208359
[26]	validation_0-logloss:0.208109

[230]	validation_0-logloss:0.083151
[231]	validation_0-logloss:0.083139
[232]	validation_0-logloss:0.083134
[233]	validation_0-logloss:0.083121
[234]	validation_0-logloss:0.083109
[235]	validation_0-logloss:0.083106
[236]	validation_0-logloss:0.082862
[237]	validation_0-logloss:0.082855
[238]	validation_0-logloss:0.082852
[239]	validation_0-logloss:0.082838
[240]	validation_0-logloss:0.082835
[241]	validation_0-logloss:0.08281
[242]	validation_0-logloss:0.082801
[243]	validation_0-logloss:0.082797
[244]	validation_0-logloss:0.08279
[245]	validation_0-logloss:0.082782
[246]	validation_0-logloss:0.082777
[247]	validation_0-logloss:0.082752
[248]	validation_0-logloss:0.082564
[249]	validation_0-logloss:0.082536
[250]	validation_0-logloss:0.082531
[251]	validation_0-logloss:0.082526
[252]	validation_0-logloss:0.082524
[253]	validation_0-logloss:0.082375
[254]	validation_0-logloss:0.082364
[255]	validation_0-logloss:0.082358
[256]	validation_0-logloss:0.082186
[257]	validation_0-logloss:0.0

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.974950461058477


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
